In [2]:
## This is Rodrigo Gutierrez's midterm notebook. Our group is researching park equity in the 2nd and 5th Supervisorial Districts
## in Los Angeles, CA. The members of my group are Annie Chan, Miles Parr Kim, Aydin Pasebani, and Max Garcia.

In [3]:
## I am focusing my code on the number of unhoused individuals in LA County comparing that to the average rent, and whether
## parks are more prevalent in census tracts with higher rent costs.

In [4]:
## Here I am importing all my libraries that I will potentially use. I may not use them all, but I like to import them just in
## case.

In [5]:
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.spatial import distance_matrix as distance_matrix
from shapely.geometry import Point

In [6]:
## Here I am reading in my GEOJSON file of the 2020 Census tracts.

In [7]:
tracts = gpd.read_file('midterm/2020_Census_Tracts.geojson')

ERROR:fiona._env:midterm/2020_Census_Tracts.geojson: No such file or directory


DriverError: midterm/2020_Census_Tracts.geojson: No such file or directory

In [ ]:
## Here I am creating a 'FIPS' column because I will use this later on to merge datasets. I used the .head() function to preview
## the data.

In [ ]:
tracts['FIPS'] ='06' + '037' + tracts['CT20']
tracts.head(3)

In [ ]:
## Here I am reading in the file containing all the information of the 2nd Supervisioral District.
## I am reading in the column 'CT20' as a string in order to create the 'FIPS' column.

In [ ]:
District2 = pd.read_csv('midterm/District2.csv',
    dtype =
    {
        'CT20':str,
    }
)

In [ ]:
District2['FIPS'] = '06' + '037' + District2['CT20']

In [ ]:
## Here I am performing my first merge with the tracts and District 2, on the column 'FIPS'. I then use .head() to visualize.

In [ ]:
Dis2tracts = tracts.merge(District2, on = 'FIPS')
Dis2tracts.head()

In [ ]:
## Reading in District 5 file, doing the same reading in 'CT20' as a string to create 'FIPS' column to merge with tracts data.

In [ ]:
District5 = pd.read_csv('midterm/District5 (1).csv',
    dtype =
    {
        'CT20':str,
    }
 )
District5

In [ ]:
District5['FIPS'] = '06' + '037' + District5['CT20']

In [ ]:
District5

In [ ]:
## Here I am merging District 5 and the tracts data on the 'FIPS' column.

In [ ]:
Dis5tracts = tracts.merge(District5, on = 'FIPS')

In [ ]:
## Previewing the data, looks like they merged good.

In [ ]:
Dis5tracts.head()

In [ ]:
## Reading in my data set with the counts of unhoused individuals in LA County. I cleaned this data in Excel before reading in,
## keeping only the columns with the tract number, location, and the counts for unsheltered, sheltered, and total population.

In [ ]:
# I read in the column 'Tract Num' as a string in order to create a 'FIPS' number, necessary if I want to merge.

In [ ]:
# I used the .head() function after to see how well the data was read, looks pretty good.

In [ ]:
unhoused_count = pd.read_csv('midterm/unhoused.csv',
    dtype =
    {
        'TractNum':str,
    }
)
unhoused_count.head()

In [ ]:
## Creating 'FIPS' number for unhoused_count, previewing it to make sure there is a leading zero. Looks good.

In [ ]:
unhoused_count['FIPS'] = '06' + '037' + unhoused_count['TractNum']
unhoused_count.head()

In [ ]:
## Performing a merge between my merged Dis2tracts and the unhoused_count on 'FIPS'. I used the .head() variable to visualize.

In [ ]:
Dis2_tracthome = Dis2tracts.merge(unhoused_count, on ='FIPS')
Dis2_tracthome.head()

In [ ]:
## Visualizing the data here, showing frequency of unhoused residents. The majority of census tracts, have less than 50.

In [ ]:
Dis2_tracthome['Total Unsheltered'].plot.hist()

In [ ]:
## Here I am doing the same merge as the line above, just doing it with my data from District 5.

In [ ]:
Dis5_tracthome = Dis5tracts.merge(unhoused_count, on ='FIPS')
Dis5_tracthome.head()

In [ ]:
## Visualizing district 5 data here with census tracts, it has a signficantly different amount of areas, but that might be
## because district 2 has more census tracts given that district 5 is mostly Antelope Valley which is more sparse.

In [ ]:
Dis5_tracthome['Total Population'].plot.hist()

In [ ]:
## I want to see what are the locations with the highest counts of unhoused individuals. I didn't know what CANNDU was,
## but a quick internet search produced that it stands for 'Community And Neighbors for Ninth District Unity'. This area is in
## South Los Angeles close to the neighborhood of Florence-Graham also nearby.

In [ ]:
Dis2_tracthome.sort_values(by='Total Population', ascending = False)[['FIPS', 'Total Population', 'Location']].head(10)

In [ ]:
## Similar to the line above, I see here that unincorporated Antelope Valley and Lancaster show up the most.

In [ ]:
Dis5_tracthome.sort_values(by='Total Population', ascending = False)[['FIPS', 'Total Population', 'Location']].head(10)

In [ ]:
## Reading in data about Gross Rent in LA County, reading in GEO_ID as a string to use it to merge. I cleaned the data in Excel.

In [ ]:
rent = pd.read_csv('midterm/rentdata.csv',
dtype=
    {
        'GEO_ID':str,
    }
)
rent

In [ ]:
## Here I am renaming the GEO_ID column to 'FIPS' in order to merge with my other data.

In [ ]:
rent['FIPS'] = rent['GEO_ID']

In [ ]:
## Checking how the data looks making sure there is a leading 0, there is!

In [ ]:
rent

In [ ]:
## Merging here rent with tracts on 'FIPS'. Looks good.

In [ ]:
tracts_rent = tracts.merge(rent,on ="FIPS")
tracts_rent

In [ ]:
## Merging here the tracts_rent with the District 2 data on 'FIPS'.

In [ ]:
tracts_rentdis2 = tracts_rent.merge(District2, on = "FIPS")

In [ ]:
## Let's see how well they combined. Looks good.

In [ ]:
tracts_rentdis2

In [ ]:
tracts_rentdis2

In [ ]:
## Found this code to take away the rows where there was no rent listed. I checked again and I'm not sure why they disappeared
## before I did this, but keeping it just in case.

In [ ]:
tracts_rentdis2 = tracts_rentdis2[tracts_rentdis2['Gross Rent'] != '-']

In [ ]:
## Here I wanted to see the top 10 highest rents. Looks like District 2's highest gross rent seems to be 3459.

In [ ]:
tracts_rentdis2.sort_values(by='Gross Rent', ascending = False)[['GEO_ID', 'Gross Rent']].head(10)

In [ ]:
## Doing the same, but for the smallest rent, seems like 675 is the smallest gross rent that could be found in a census tract.

In [ ]:
tracts_rentdis2.sort_values(by='Gross Rent', ascending = False)[['GEO_ID', 'Gross Rent']].tail(5)

In [ ]:
## Checking if any of the columns are empty. Doesn't appear like there is.

In [ ]:
tracts_rentdis2.columns[tracts_rentdis2.isna().all()].tolist()

In [ ]:
## I had tried to plot this, but it said 'Gross Rent' wasn't plotable because they weren't integers, found this code to change it.

In [ ]:
tracts_rentdis2['Gross Rent'] = pd.to_numeric(tracts_rentdis2['Gross Rent'], errors='coerce')

In [ ]:
## Using this next code to drop any NaN values that I do not want to plot within the 'Gross Rent' column.

In [ ]:
tracts_rentdis2.dropna(subset=['Gross Rent'], inplace=True)

In [ ]:
## Time to plot this finally, let's see if it works. First let's see how it looks on a histogram. Seems like the majority of
## tracts fall between 1000 and 1500 rent.

In [ ]:
tracts_rentdis2['Gross Rent'].plot.hist()

In [ ]:
## Now let's see it on a better map.

In [ ]:
dis2_tractsrentmap = tracts_rentdis2.plot(figsize=(12,10),
                 column='Gross Rent',
                 legend=True,
                 scheme='NaturalBreaks')
dis2_tractsrentmap.set_title('Gross Rent Costs in District 2 of LA County')
dis2_tractsrentmap.axis('off')

In [ ]:
## Map looks pretty good, we can see that higher rent tends to be more on the Westside of the District, lower rent is more
## on Eastside.

In [ ]:
## Here I am not merging my District 5 data with my tracts_rent data, doing so on 'FIPS'.

In [ ]:
tracts_rentdis5 = tracts_rent.merge(District5, on = "FIPS")

In [ ]:
## Similar to above, I am using this code to get rid of any values showing up as '-' in the 'Gross Rent' column.

In [ ]:
tracts_rentdis5 = tracts_rentdis5[tracts_rentdis5['Gross Rent'] != '-']

In [ ]:
## Also like above, converting the values in 'Gross Rent' into integers.

In [ ]:
tracts_rentdis5['Gross Rent'] = pd.to_numeric(tracts_rentdis5['Gross Rent'], errors='coerce')

In [ ]:
## Dropping the NaN values in the 'Gross Rent' column.

In [ ]:
tracts_rentdis5.dropna(subset=['Gross Rent'], inplace=True)

In [ ]:
## Time to plot the data again, using same code as before. This graph looks different than District 2's, more census tracts
## fall between 1500 and 2000 here.

In [ ]:
tracts_rentdis5['Gross Rent'].plot.hist()

In [ ]:
## Now let's map it on a better map similar to the one above.

In [ ]:
dis5_tractrentmap = tracts_rentdis5.plot(figsize=(12,10),
                 column='Gross Rent',
                 legend=True,
                 scheme='NaturalBreaks')
dis5_tractrentmap.set_title('Gross Rent Costs in District 5 of LA County')
dis5_tractrentmap.axis('off')

In [ ]:
### Looks good! Here we can see the Census Tracts are much bigger, because population is so sparse, in the more densely
### populated tracts, we see higher rents.

In [ ]:
## Now lets read in the Parks data from the County.

In [ ]:
parks = gpd.read_file('midterm/Parks.geojson')
parks.head()

In [ ]:
## Here I am merging my Parks data with my data of the Rent in District 2 tracts, on the column of 'Polygon'.

In [ ]:
from shapely.geometry import Polygon
geopandas = gpd
parks_locationsD2 = geopandas.overlay(tracts_rentdis2, parks, how='intersection', keep_geom_type=None, make_valid=True)
parks_locationsD2

In [ ]:
## Now let's map this.

In [ ]:
dis2_tractsrentmap = parks_locationsD2.plot(figsize=(12,10),
                            column='Gross Rent',
                 legend=True,
                 scheme='NaturalBreaks')
dis2_tractsrentmap.set_title('Parks in relation to Rents in their Census Tracts in LA Supervisorial District 2')
plt.savefig('D2_Rent_Parks_Pct.jpg')
plt.show()

In [ ]:
## Now let's do the same for District 5, merging on "Polygon'.

In [ ]:
parks_locationsD5 = geopandas.overlay(tracts_rentdis5, parks, how='intersection', keep_geom_type=None, make_valid=True)

In [ ]:
## Let's map this the same as before, for District 5.

In [ ]:
dis5_tractsrentmap = parks_locationsD5.plot(figsize=(12,10),
                            column='Gross Rent',
                 legend=True,
                 scheme='NaturalBreaks')
dis5_tractsrentmap.set_title('Parks in relation to Rents in their Census Tracts in LA Supervisorial District 5')
plt.savefig('D5_Rent_Parks_Pct.jpg')
plt.show()

In [ ]:
## This is interesting, there is a very large park, most likely a national park. It borders a park in an area with the highest
## rent. We can see that the large park may have a low rent because few people actually live there. Lots of parks can be found
## in Southern part of the District, it is more densely populated.

In [ ]:
## Now lets map the unhoused Population in District 2.

In [ ]:
Dis2_tractshomemap = Dis2_tracthome.plot(figsize=(12,10),
                 column='Total Population',
                 legend=True,
                 scheme='NaturalBreaks')
Dis2_tractshomemap.set_title('Total Unhoused Population in District 2 of LA County')
Dis2_tractshomemap.axis('off')

In [ ]:
## Now let's do the same as with District 2, but this time with District 5.

In [ ]:
Dis5_tractshomemap = Dis5_tracthome.plot(figsize=(12,10),
                 column='Total Population',
                 legend=True,
                 scheme='NaturalBreaks')
Dis5_tractshomemap.set_title('Total Unhoused Population in District 5 of LA County')
Dis5_tractshomemap.axis('off')

In [ ]:
## Now let's merge Parks data with Homeless data like we did with the rent. We are merging on the Polygon.

In [ ]:
parks_locationshomeD2 = geopandas.overlay(Dis2_tracthome, parks, how='intersection', keep_geom_type=None, make_valid=True)

In [ ]:
## Now we can map this like we did with the rent. Interesting, the bigger parks seem to have more unhoused individuals with one small one having the most.

In [ ]:
dis2_tractshomemap = parks_locationshomeD2.plot(figsize=(12,10),
                            column='Total Population',
                 legend=True,
                 scheme='NaturalBreaks')
dis2_tractshomemap.set_title('Parks in relation to Total Unhoused in their Census Tracts in LA Supervisorial District 2')
plt.savefig('D2_Home_Parks_Pct.jpg')
plt.show()

In [ ]:
## Now lets do the same with District 5, merging parks with homeless count.

In [ ]:
parks_locationshomeD5 = geopandas.overlay(Dis5_tracthome, parks, how='intersection', keep_geom_type=None, make_valid=True)
parks_locationshomeD5

In [ ]:
## Now we can map it like we did before, interesting that the parks in the Northeast corner hold a lot of unhoused residents.

In [ ]:
dis5_tractshomemap = parks_locationshomeD5.plot(figsize=(12,10),
                            column='Total Population',
                 legend=True,
                 scheme='NaturalBreaks')
dis5_tractshomemap.set_title('Parks in relation to Total Unhoused in their Census Tracts in LA Supervisorial District 5')
plt.savefig('D5_Home_Parks_Pct.jpg')
plt.show()

In [ ]:
## Now let's import folium and try to visualize the data. I'm not a fan of this map, it is not very helpful.

In [ ]:
import folium

In [ ]:
mymap = folium.Map(location=[34.2,-118.2], zoom_start=10)  # Los Angeles coordinates

# Add GeoJSON data to the map
folium.GeoJson(
    parks_locationshomeD5,
    name='Parks',
).add_to(mymap)

legend_html = '''
     <div style="position: fixed;
     bottom: 50px; left: 50px; width: 150px; height: 120px;
     background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
     ">&nbsp; Parks <br>
     &nbsp; <img src="https://via.placeholder.com/10/00FF00"> &nbsp; Park
     <br>
     &nbsp; <img src="https://via.placeholder.com/10/FF0000"> &nbsp; Another Park
      </div>
     '''

mymap.get_root().html.add_child(folium.Element(legend_html))
# Display the map
mymap

In [ ]:
## Let's try another way, using Chris' code from the lecture. Looks good, now to repeat for District 5.

In [ ]:
m = folium.Map(location=[34.2,-118.2],
               zoom_start = 9,
               tiles='CartoDB positron',
               attribution='CartoDB')

# plot chorpleth over the base map
folium.Choropleth(
                  geo_data=tracts_rentdis2 , # geo data
                  data=tracts_rentdis2 , # data
                  key_on='feature.properties.FIPS', # key, or merge column
                  columns=['FIPS', 'Gross Rent'], # [key, value]
                  fill_color='BuPu',
                  line_weight=0.1,
                  fill_opacity=0.8,
                  line_opacity=0.2, # line opacity (of the border)
                  legend_name='Gross Rent').add_to(m)    # name on the legend color bar
m

In [ ]:
m.save('tracts_home2.html')

In [ ]:
## Did the same thing here with District 5, this is good for helping visualize and compare rents across areas.

In [ ]:
m = folium.Map(location=[34.2,-118.2],
               zoom_start = 9,
               tiles='CartoDB positron',
               attribution='CartoDB')

# plot chorpleth over the base map
folium.Choropleth(
                  geo_data=tracts_rentdis5 , # geo data
                  data=tracts_rentdis5 , # data
                  key_on='feature.properties.FIPS', # key, or merge column
                  columns=['FIPS', 'Gross Rent'], # [key, value]
                  fill_color='BuPu',
                  line_weight=0.1,
                  fill_opacity=0.8,
                  line_opacity=0.2, # line opacity (of the border)
                  legend_name='Gross Rent').add_to(m)    # name on the legend color bar
m

In [ ]:
m.save('tracts_home5.html')

In [ ]:
## Now we can repeat but this time with the homeless population in District 2.

In [ ]:
m = folium.Map(location=[34.2,-118.2],
               zoom_start = 9,
               tiles='CartoDB positron',
               attribution='CartoDB')

# plot chorpleth over the base map
folium.Choropleth(
                  geo_data=Dis2_tracthome , # geo data
                  data=Dis2_tracthome , # data
                  key_on='feature.properties.FIPS', # key, or merge column
                  columns=['FIPS', 'Total Population'], # [key, value]
                  fill_color='BuPu',
                  line_weight=0.1,
                  fill_opacity=0.8,
                  line_opacity=0.2, # line opacity (of the border)
                  legend_name='Total Population').add_to(m)    # name on the legend color bar
m

In [ ]:
m.save('tracts_home2.html')

In [ ]:
## Now doing it again, but with District 5. Looks good.

In [ ]:
m = folium.Map(location=[34.2,-118.2],
               zoom_start = 9,
               tiles='CartoDB positron',
               attribution='CartoDB')

# plot chorpleth over the base map
folium.Choropleth(
                  geo_data=Dis5_tracthome , # geo data
                  data=Dis5_tracthome , # data
                  key_on='feature.properties.FIPS', # key, or merge column
                  columns=['FIPS', 'Total Population'], # [key, value]
                  fill_color='BuPu',
                  line_weight=0.1,
                  fill_opacity=0.8,
                  line_opacity=0.2, # line opacity (of the border)
                  legend_name='Total Population').add_to(m)    # name on the legend color bar
m

In [ ]:
m.save('tracts_home5.html')

In [ ]:
## Now let's do it with the merged homeless count and parks data. Very weird, I don't see anything showing for the 2nd District, let's try the 5th.

In [ ]:
import folium

# Initialize Folium map
m = folium.Map(location=[34.2, -118.2], zoom_start=9, tiles='CartoDB positron', attribution='CartoDB')

# Plot choropleth over the base map
folium.Choropleth(
    geo_data=parks_locationshomeD2,  # GeoJSON or GeoDataFrame
    data=parks_locationshomeD2,       # Dataframe or Series
    key_on='feature.properties.FIPS',  # GeoJSON property to bind the data to
    columns=['FIPS', 'Total Population'],  # Column(s) in the dataframe to use
    fill_color='BuPu',  # Color scale
    line_weight=0.1,
    fill_opacity=0.8,
    line_opacity=0.2,
    legend_name='<Unhoused Population near Parks>'  # Name on the legend color bar
).add_to(m)
m

In [ ]:
m.save('park_homeless2.html')

In [ ]:
## Huh, interesting, it does show up on this map, we can see the parks and the number of unhoused.

In [ ]:
import folium

# Initialize Folium map
m = folium.Map(location=[34.2, -118.2], zoom_start=9, tiles='CartoDB positron', attribution='CartoDB')

# Plot choropleth over the base map
folium.Choropleth(
    geo_data=parks_locationshomeD5,  # GeoJSON or GeoDataFrame
    data=parks_locationshomeD5,       # Dataframe or Series
    key_on='feature.properties.FIPS',  # GeoJSON property to bind the data to
    columns=['FIPS', 'Total Population'],  # Column(s) in the dataframe to use
    fill_color='BuPu',  # Color scale
    line_weight=0.1,
    fill_opacity=0.8,
    line_opacity=0.2,
    legend_name='<Unhoused near Parks>'  # Name on the legend color bar
).add_to(m)
m

In [ ]:
m.save('park_homeless5.html')

In [ ]:
## Now lets do the data of parks with the gross rents. Here is District 2.

In [ ]:
import folium

# Initialize Folium map
m = folium.Map(location=[34.2, -118.2], zoom_start=9, tiles='CartoDB positron', attribution='CartoDB')

# Plot choropleth over the base map
folium.Choropleth(
    geo_data=parks_locationsD2,  # GeoJSON or GeoDataFrame
    data=parks_locationsD2,       # Dataframe or Series
    key_on='feature.properties.FIPS',  # GeoJSON property to bind the data to
    columns=['FIPS', 'Gross Rent'],  # Column(s) in the dataframe to use
    fill_color='BuPu',  # Color scale
    line_weight=0.1,
    fill_opacity=0.8,
    line_opacity=0.2,
    legend_name='Rent in District 2'  # Name on the legend color bar
).add_to(m)
m

In [ ]:
m.save('park_rent2.html')

In [ ]:
## Here is District 5, with parks and gross rent.

In [ ]:
import folium

# Initialize Folium map
m = folium.Map(location=[34.2, -118.2], zoom_start=9, tiles='CartoDB positron', attribution='CartoDB')

# Plot choropleth over the base map
folium.Choropleth(
    geo_data=parks_locationsD5,  # GeoJSON or GeoDataFrame
    data=parks_locationsD5,       # Dataframe or Series
    key_on='feature.properties.FIPS',  # GeoJSON property to bind the data to
    columns=['FIPS', 'Gross Rent'],  # Column(s) in the dataframe to use
    fill_color='BuPu',  # Color scale
    line_weight=0.1,
    fill_opacity=0.8,
    line_opacity=0.2,
    legend_name='Rent in District 5'  # Name on the legend color bar
).add_to(m)
m

In [ ]:
m.save('park_rent5.html')